## Notebook Overview

Take the data from notebook 1, and flatten the nested data structures so it can be put into sql-database-friendly form.

Belatedly noticing that in some sports, they list the coaches as a second table... decide later whether we want to gather that as well.



In [1]:
# Use w/ Anaconda Distribution
# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

# Backoff time for large scrapes
THROTTLE_TIME = .05

# ipython magics
%load_ext watermark
%matplotlib inline

In [2]:
import missingno as msno

In [4]:
%watermark -a "Cameron Yick" -d -t -v -p pandas,seaborn,matplotlib,bs4 -g

Cameron Yick 2016-11-05 21:41:16 

CPython 2.7.12
IPython 4.0.1

pandas 0.17.1
seaborn 0.7.1
matplotlib 1.5.0
bs4 4.4.1
Git hash: ed1904a929cd8dcb812e0a7193202e2ab046079e


In [5]:
RAW_PATH = os.path.join(PROJ_ROOT, "data", "raw", "sports.json")
RAW_PICK = os.path.join(PROJ_ROOT, "data", "raw", "sports.p")

In [6]:
sports = pd.read_pickle(RAW_PICK)

## Can we check how the size of a roster has changed each year, for each sport?

A roster is a collection of players for 1 season.

In [7]:
sports.columns

Index([u'href', u'name', u'gender', u'currentRoster', u'seasons', u'nSeasons',
       u'rosters'],
      dtype='object')

We have 33 collections of rosters, 1 for each sport

In [8]:
sports.rosters

0     {u'2008-09': [[{'link': '/sports/m-basebl/2008...
1     {u'2008-09': [[{'link': '/sports/m-baskbl/2008...
2     {u'2008-09': [[{'link': '/sports/m-crewhvy/200...
3     {u'2012-13': [[{'link': '/sports/m-crewlt/2012...
4     {u'2012-13': [[{'link': '/sports/m-xc/2012-13/...
5     {u'2008-09': [[{'link': '/sports/m-fenc/2008-0...
6     {u'2008-09': [[{'link': '/sports/m-footbl/2008...
7     {u'2008-09': [[{'link': '/sports/m-golf/2008-0...
8     {u'2012-13': [[{'link': '/sports/m-hockey/2012...
9     {u'2012-13': [[{'link': '/sports/m-lacros/2012...
10    {u'2008-09': [[{'link': '/sports/c-sail/2008-0...
11    {u'2008-09': [[{'link': '/sports/m-soccer/2008...
12    {u'2012-13': [[{'link': '/sports/m-squash/2012...
13    {u'2012-13': [[{'link': '/sports/m-swim/2012-1...
14    {u'2012-13': [[{'link': '/sports/m-tennis/2012...
15    {u'2008-09': [[{'link': '/sports/m-track/2008-...
16    {u'2008-09': [[{'link': '/sports/w-baskbl/2008...
17    {u'2012-13': [[{'link': '/sports/w-crew/20

In [9]:
# The top entry for every table describes what metadata is available for players in that table.
sports.iloc[0]['rosters']['2016-17'][-1:]

[[u'',
  u'',
  {'link': '/sports/m-basebl/2016-17/bios/stiegler_alex_3dje',
   'name': u'Alex Stiegler'},
  u'RHP/SS',
  u'Freshman',
  u'R/R',
  u'',
  u'',
  u'- / -']]

Before we can do anything fun, we need to attach these labels as keys onto the cell row entries.

In [10]:
def getFieldnames(rows):
    'Given an array of rows from a table, return a list of what to name all the keys in that table.'
    header = rows[0]
    names = []
    for fieldName in header:
        if type(fieldName) is dict:
            names.append(fieldName['name'])
        else:
            names.append(fieldName)
            
    return names

In [11]:
rosters = []

for i, row in sports.iterrows(): # for each sport
    sportName = row['name']
    
    for season, roster in row['rosters'].iteritems():    # check for each season
        nRoster = {}
        players = [] # new array of players for every season
        
        fieldNames = getFieldnames(roster)
           
        nRoster['sport'] = sportName             # this is the foreign key, when combined with the season
        nRoster['season'] = season
        nRoster['metadata'] = fieldNames
       
        #         print "{0} {1} {2}".format(season, sportName, len(roster[1:]))
        
        for person in roster[1:]: # iterate throw the tail rows of the roster, and drop the last entry
            nPlayer = {}
            nPlayer['sport'] = sportName
            nPlayer['season'] = season

            for i, val in enumerate(fieldNames):
                
                fieldVal = person[i+1] # do to skip the headshot column
                
                if type(fieldVal) is dict:
                    nPlayer[val + '_link'] = fieldVal['link']
                    nPlayer[val] = fieldVal['name']
                else:
                    nPlayer[val] = fieldVal
                    
            players.append(nPlayer)
        
        nRoster['players'] = players
        
        rosters.append(nRoster)
        

In [12]:
len(rosters)

328

In [13]:
players = []

for roster in rosters:
    players.extend(roster['players'])

In [14]:
len(players)

8702

In [15]:
pdf = pd.DataFrame(players)

In [16]:
print pdf.columns
pdf.shape

Index([                 u'B/T',                  u'Cl.',
                     u'Events', u'Hometown/High School',
                        u'Ht.',                 u'Name',
                  u'Name_link',                  u'No.',
                       u'Pos.',               u'Weapon',
                        u'Wt.',               u'season',
                      u'sport'],
      dtype='object')


(8702, 13)

In [17]:
pdf.head()

,B/T,Cl.,Events,Hometown/High School,Ht.,Name,Name_link,No.,Pos.,Weapon,Wt.,season,sport
0,,Freshman,NaN,"Manhasset, N.Y. / Chaminade",5-11,Ryan Brenner,/sports/m-basebl/mtt/brenner_ryan00.html,11,C,NaN,180,2008-09,Baseball
1,,Junior,NaN,"Sunrise, Fla. / Pine Crest School",6-2,Joe Castaldi,/sports/m-basebl/mtt/castaldi_joe00.html,34,P,NaN,190,2008-09,Baseball
2,,Sophomore,NaN,"Bloomington, Ind. / Bloomington North",5-9,Gant Elmore,/sports/m-basebl/mtt/elmore_gant00.html,1,IF,NaN,170,2008-09,Baseball
3,,Junior,NaN,"Suffern, N.Y. / Don Bosco Prep",5-11,Chris Finneran,/sports/m-basebl/mtt/finneran_chris00.html,40,P,NaN,175,2008-09,Baseball
4,,Junior,NaN,"Atlanta, Ga. / Chamblee",6-7,Robert Gruber,/sports/m-basebl/mtt/gruber_robert00.html,29,P,NaN,220,2008-09,Baseball


In [18]:
# beware inconsistency in high school name
pdf[pdf['Name'] == 'Emily Waligurski']

,B/T,Cl.,Events,Hometown/High School,Ht.,Name,Name_link,No.,Pos.,Weapon,Wt.,season,sport
5813,NaN,So.,NaN,"Hurley, N.Y. / Onteora",NaN,Emily Waligurski,/sports/w-xc/2014-15/bios/waligurski_emily_5lkl,NaN,NaN,NaN,NaN,2014-15,Cross Country
5869,NaN,Sr.,NaN,"Hurley, N.Y. / Onteora High School",NaN,Emily Waligurski,/sports/w-xc/2016-17/bios/waligurski_emily_unhf,NaN,NaN,NaN,NaN,2016-17,Cross Country
5950,NaN,Jr.,NaN,"Hurley, N.Y. / Onteora High School",NaN,Emily Waligurski,/sports/w-xc/2015-16/bios/waligurski_emily_9wvy,NaN,NaN,NaN,NaN,2015-16,Cross Country
6001,NaN,Fr.,NaN,"Hurley, N.Y. / Onteora",NaN,Emily Waligurski,/sports/w-xc/2013-14/bios/waligurski_emily_hhnf,NaN,NaN,NaN,NaN,2013-14,Cross Country
8221,NaN,Fr.,Distance,"Hurley, N.Y. / Onteora High School",NaN,Emily Waligurski,/sports/w-track/2012-13/bios/waligurski_emily_...,NaN,NaN,NaN,NaN,2012-13,Track & Field
8266,NaN,So.,Distance,"Hurley, N.Y. / Onteora Highschool",NaN,Emily Waligurski,/sports/w-track/2014-15/bios/waligurski_emily_...,NaN,NaN,NaN,NaN,2014-15,Track & Field
8313,NaN,Sr.,Distance,"Hurley, N.Y. / Onteora",NaN,Emily Waligurski,/sports/w-track/2016-17/bios/waligurski_emily_...,NaN,NaN,NaN,NaN,2016-17,Track & Field
8355,NaN,Jr.,Distance,"Hurley, N.Y. / Onteora",NaN,Emily Waligurski,/sports/w-track/2015-16/bios/waligurski_emily_...,NaN,NaN,NaN,NaN,2015-16,Track & Field
8448,NaN,Fr.,Distance,"Hurley, N.Y. / Onteora Highschool",NaN,Emily Waligurski,/sports/w-track/2013-14/bios/waligurski_emily_...,NaN,NaN,NaN,NaN,2013-14,Track & Field


In [19]:
# check which players are doing stuff every semester
pdf.groupby('Name').count().sort_values('Cl.', ascending=False).head(10)

,B/T,Cl.,Events,Hometown/High School,Ht.,Name_link,No.,Pos.,Weapon,Wt.,season,sport
Name,,,,,,,,,,,,
Emily Waligurski,0,9,5,9,0,9,0,0,0,0,9,9
Meredith Rizzo,0,9,5,9,0,9,0,0,0,0,9,9
Delaney FitzPatrick,0,8,4,8,0,8,0,0,0,0,8,8
Amanda Salvesen,0,8,0,8,0,8,0,0,0,0,8,8
Elizabeth McDonald,0,8,4,8,0,8,0,0,0,0,8,8
Andre Ivankovic,0,8,4,8,0,8,0,0,0,0,8,8
Chandler Olson,0,8,4,8,0,8,0,0,0,0,8,8
Duncan Tomlin,0,8,4,8,0,8,0,0,0,0,8,8
Jacob Sandry,0,8,4,8,0,8,0,0,0,0,8,8


In [20]:
def lookupPlayer(name):
    return pdf[pdf['Name'] == name]

In [21]:
# Who is a multi sport athlete?
pdf.groupby('Name')['sport'].nunique().sort_values(ascending=False).head(10)

Name
Lindsey Raymond    3
Jacque Sahlberg    3
Conor Dooney       2
Matt Chisholm      2
Hannah Steffke     2
Matt Bogdan        2
Chris Stanley      2
Sarah Smith        2
Hannah Smith       2
Kristin Johnson    2
Name: sport, dtype: int64

In [22]:
# how many of these people are there?!
multis = pdf.groupby('Name')['sport'].nunique().sort_values(ascending=False)

# there were 246 multisport athletes! Their names can be 
multis = multis[multis >= 2]

In [23]:
multis.index

Index([u'Lindsey Raymond', u'Jacque Sahlberg', u'Conor Dooney',
       u'Matt Chisholm', u'Hannah Steffke', u'Matt Bogdan', u'Chris Stanley',
       u'Sarah Smith', u'Hannah Smith', u'Kristin Johnson',
       ...
       u'Max Brown', u'Juan Wheat', u'Nathan Richards', u'Jeff Perrella',
       u'Morgan Kiss', u'Urska Kosir', u'Grace Zimmerly', u'Alexa Chu',
       u'Claire Huebner', u'Grace Brittan'],
      dtype='object', name=u'Name', length=246)

In [24]:
# we need to make a separate table for when each of these people started yale so you can ask if there's a pattern to this!

In [25]:
lookupPlayer("Lindsey Raymond")

,B/T,Cl.,Events,Hometown/High School,Ht.,Name,Name_link,No.,Pos.,Weapon,Wt.,season,sport
5977,NaN,Sr.,NaN,"Acton, Mass. / Acton-Boxborough Regional High ...",NaN,Lindsey Raymond,/sports/w-xc/2011-12/bios/raymond_lindsey_ojqj,NaN,NaN,NaN,NaN,2011-12,Cross Country
7272,NaN,Freshman,NaN,"Acton, MA / Acton Boxborough",5-8,Lindsey Raymond,/sports/w-soccer/mtt/raymond_lindsey00.html,19,M,NaN,NaN,2008-09,Soccer
8396,NaN,Sr.,Distance,"Acton, Mass. / Acton-Boxborough Regional High ...",NaN,Lindsey Raymond,/sports/w-track/2011-12/bios/raymond_lindsey_fgqe,NaN,NaN,NaN,NaN,2011-12,Track & Field


In [26]:
# We know his height from one sport
# he has different bios in different sports.
# this table is actually a "relational" table called "played". Every time someone plays in a season, they get an entry here.

# note that if the play every year, their bio might change from year to year. in some cases their major is added in their oldest bio.

lookupPlayer("Jacque Sahlberg")

,B/T,Cl.,Events,Hometown/High School,Ht.,Name,Name_link,No.,Pos.,Weapon,Wt.,season,sport
5730,NaN,Sr.,NaN,"Boise, Idaho / Boise",NaN,Jacque Sahlberg,/sports/w-xc/2012-13/bios/sahlberg_jacque_d63t,NaN,NaN,NaN,NaN,2012-13,Cross Country
5979,NaN,Jr.,NaN,"Boise, Idaho / Boise",NaN,Jacque Sahlberg,/sports/w-xc/2011-12/bios/sahlberg_jacque_kiii,NaN,NaN,NaN,NaN,2011-12,Cross Country
6024,NaN,So.,NaN,"Boise, Idaho / Boise",NaN,Jacque Sahlberg,/sports/w-xc/2010-11/bios/sahlberg_jacque_kiii,NaN,NaN,NaN,NaN,2010-11,Cross Country
7275,NaN,Freshman,NaN,"Boise, ID / Boise",5-5,Jacque Sahlberg,/sports/w-soccer/mtt/sahlberg_jacque00.html,24,D,NaN,NaN,2008-09,Soccer
7482,NaN,So.,NaN,"Boise, ID / Boise",5-5,Jacque Sahlberg,/sports/w-soccer/2009-10/bios/sahlberg_jacque0...,24,D,NaN,NaN,2009-10,Soccer
8400,NaN,Sr.,Distance,"Boise, Idaho / Boise",NaN,Jacque Sahlberg,/sports/w-track/2011-12/bios/sahlberg_jacque_g31m,NaN,NaN,NaN,NaN,2011-12,Track & Field
8487,NaN,Jr.,Distance,"Boise, Idaho / Boise",NaN,Jacque Sahlberg,/sports/w-track/2010-11/bios/sahlberg_jacque_g31m,NaN,NaN,NaN,NaN,2010-11,Track & Field


In [27]:
pdf['Hometown/Region'], pdf['High School'] = zip(*pdf['Hometown/High School'].apply(lambda x: x.split(" / ")))

In [28]:
def splitCityReg(string):
    if string == "-":
        return ["-", "-"]
    else:
        
        pair = string.split(", ")
        if len(pair) is 2:
            return pair
        else:
            return [string,'NOREGION?']

In [29]:
pdf['City'], pdf['Region'] = zip(*pdf['Hometown/Region'].apply(lambda x: splitCityReg(x) ))

In [30]:
m_cities_regs = pdf.groupby('Hometown/Region')['Name'].nunique().sort_values(ascending=False)
m_cities_regs

Hometown/Region
New York, N.Y.                 45
Greenwich, Conn.               25
-                              24
Los Angeles, Calif.            22
Dallas, TX                     21
Washington, D.C.               19
San Francisco, Calif.          17
San Diego, Calif.              16
Baltimore, Md.                 15
Boston, Mass.                  15
Los Angeles, CA                13
Seattle, WA                    13
Seattle, Wash.                 12
Houston, Texas                 12
Pittsburgh, PA                 11
St. Louis, Mo.                 11
Newport Beach, Calif.          11
Portland, OR                   11
Cincinnati, OH                 11
Greenwich, CT                  10
Darien, Conn.                  10
Newton, Mass.                  10
Dallas, Texas                  10
London, United Kingdom          9
Manhasset, N.Y.                 9
Palo Alto, Calif.               9
Bronxville, N.Y.                9
New Haven, Conn.                9
Alexandria, VA                  

In [31]:
m_school = pdf.groupby('High School')['Name'].nunique().sort_values(ascending=False)
m_school

High School
-                                  44
Deerfield Academy                  19
New Trier                          18
Chaminade                          18
Phillips Exeter                    16
Phillips Exeter Academy            16
Choate Rosemary Hall               13
Greenwich Academy                  12
The Lawrenceville School           12
The Pingry School                  12
University                         12
Buckingham Browne & Nichols        11
Manhasset                          11
Harvard-Westlake                   10
Phillips Academy                   10
St. Ignatius                        9
Bronxville                          9
Thomas Jefferson                    9
Brunswick                           8
Deerfield                           8
Darien                              8
Harvard Westlake                    8
Hopkins School                      8
Hotchkiss School                    8
Phillips Academy Andover            8
Lawrenceville                       8


In [32]:
pdf['Hometown/Region'][0].split(",")

[u'Manhasset', u' N.Y.']

In [33]:
m_city = pdf.groupby('City')['Name'].nunique().sort_values(ascending=False)
m_city

City
New York               53
Los Angeles            35
Greenwich              31
Dallas                 26
Seattle                25
-                      24
San Diego              24
Washington             22
London                 21
San Francisco          21
Houston                19
Baltimore              17
Portland               17
Boston                 16
St. Louis              16
Darien                 16
Weston                 15
Pittsburgh             14
Palo Alto              13
Nashville              13
Bronxville             13
Newton                 13
Chicago                13
Cincinnati             13
Newport Beach          13
Bethesda               13
Honolulu               12
Alexandria             12
Garden City            11
Tampa                  11
                       ..
Nashua                  1
Nassau                  1
North Babylon           1
Norman                  1
Norfolk                 1
Nolensville             1
Nobleton                1
Nivot  

In [34]:
m_reg = pdf.groupby('Region')['Name'].nunique().sort_values(ascending=False)
m_reg

Region
Calif.           262
N.Y.             200
CA               175
Mass.            141
N.J.             137
Conn.            123
CT               112
NY               107
Pa.               82
NJ                81
Ill.              76
MA                75
TX                74
Md.               67
PA                65
Fla.              61
IL                60
Texas             52
Va.               50
FL                49
OH                45
MD                44
WA                40
Wash.             39
Mich.             37
NOREGION?         34
VA                30
Ont.              26
MI                26
-                 24
                ... 
Israel             1
Calif              1
Nigeria            1
Kans.              1
MAN                1
Il.                1
MT                 1
Ma.                1
I.L.               1
Massachusetts      1
Mexico             1
Miss.              1
N.S.               1
Florida            1
New York           1
F.L.               1
Califo

In [35]:
pdf[pdf['Region'] == "NOREGION?"].head(20)

,B/T,Cl.,Events,Hometown/High School,Ht.,Name,Name_link,No.,Pos.,Weapon,Wt.,season,sport,Hometown/Region,High School,City,Region
861,NaN,So.,NaN,"St. Catharines, Ont., Canada / Governor Simcoe",NaN,Aaron Jones,/sports/m-crewlt/2014-15/bios/jones_aaron_shcw,NaN,NaN,NaN,NaN,2014-15,Crew (Lightweight),"St. Catharines, Ont., Canada",Governor Simcoe,"St. Catharines, Ont., Canada",NOREGION?
894,NaN,Sr.,NaN,"St. Catharines, Ont., Canada / Governor Simcoe",NaN,Aaron Jones,/sports/m-crewlt/2016-17/bios/jones_aaron_atfu,NaN,NaN,NaN,NaN,2016-17,Crew (Lightweight),"St. Catharines, Ont., Canada",Governor Simcoe,"St. Catharines, Ont., Canada",NOREGION?
985,NaN,Jr.,NaN,"St. Catharines, Ont., Canada / Governor Simcoe",NaN,Aaron Jones,/sports/m-crewlt/2015-16/bios/jones_aaron_8tmt,NaN,NaN,NaN,NaN,2015-16,Crew (Lightweight),"St. Catharines, Ont., Canada",Governor Simcoe,"St. Catharines, Ont., Canada",NOREGION?
1069,NaN,Fr.,NaN,"St. Catharines, Ont., Canada / Governor Simcoe",NaN,Aaron Jones,/sports/m-crewlt/2013-14/bios/jones_aaron_g4f2,NaN,NaN,NaN,NaN,2013-14,Crew (Lightweight),"St. Catharines, Ont., Canada",Governor Simcoe,"St. Catharines, Ont., Canada",NOREGION?
1100,NaN,Jr.,NaN,Lebanon. N.H. / International School of Manila,NaN,Regina DeLuna,/sports/m-crewlt/2010-11/bios/deluna_regina_zexq,NaN,NaN,NaN,NaN,2010-11,Crew (Lightweight),Lebanon. N.H.,International School of Manila,Lebanon. N.H.,NOREGION?
1203,NaN,Fr.,NaN,San Diego / University City High School,NaN,Allen Siegler,/sports/m-xc/2016-17/bios/siegler_allen_p2b5,NaN,NaN,NaN,NaN,2016-17,Cross Country,San Diego,University City High School,San Diego,NOREGION?
1335,NaN,Freshman,NaN,New York / Bronx High School of Science,NaN,Jin Ishizuka,/sports/m-fenc/2012-13/bios/ishizuka_jin_zr5g,NaN,NaN,Foil,NaN,2012-13,Fencing,New York,Bronx High School of Science,New York,NOREGION?
1349,NaN,Junior,NaN,New York / Bronx High School of Science,NaN,Jin Ishizuka,/sports/m-fenc/2014-15/bios/ishizuka_jin_v90n,NaN,NaN,Foil,NaN,2014-15,Fencing,New York,Bronx High School of Science,New York,NOREGION?
1352,NaN,Sophomore,NaN,Los Angeles / Loyola High School,NaN,Frederick Muth,/sports/m-fenc/2014-15/bios/muth_frederick_kkp1,NaN,NaN,Épée,NaN,2014-15,Fencing,Los Angeles,Loyola High School,Los Angeles,NOREGION?
1358,NaN,Sophomore,NaN,San Francisco / San Francisco University High ...,NaN,Jacob Stein,/sports/m-fenc/2014-15/bios/stein_jacob_ntuq,NaN,NaN,"Foil, Sabre",NaN,2014-15,Fencing,San Francisco,San Francisco University High School,San Francisco,NOREGION?


In [36]:
# if you're in canada, you get your country and your province
# sometimes people just leave the state off.

# this data should probably be cleaned by hand!
# people from big cities tend to leave the state off

In [37]:
EXPORT_PLAYERS = os.path.join(PROJ_ROOT, 'data', 'interim', 'players.json')
EXPORT_ROSTERS = os.path.join(PROJ_ROOT, 'data', 'interim', 'rosters.p')

In [38]:
pdf.to_json(EXPORT_PLAYERS)

In [182]:
rdf = pd.DataFrame(rosters)

In [180]:
# verify later that tables are the same across all years for any given sport.
rdf.shape
rdf.head()

,metadata,players,season,sport
0,"[No., Name, Pos., Cl., B/T, Ht., Wt., Hometown...","[{u'Pos.': u'C', u'No.': u'11', u'Name_link': ...",2008-09,Baseball
1,"[No., Name, Pos., Cl., B/T, Ht., Wt., Hometown...","[{u'Pos.': u'IF/OF', u'No.': u'4', u'Name_link...",2012-13,Baseball
2,"[No., Name, Pos., Cl., B/T, Ht., Wt., Hometown...","[{u'Pos.': u'RHP', u'No.': u'12', u'Name_link'...",2014-15,Baseball
3,"[No., Name, Pos., Cl., B/T, Ht., Wt., Hometown...","[{u'Pos.': u'IF/OF', u'No.': u'1', u'Name_link...",2016-17,Baseball
4,"[No., Name, Pos., Cl., B/T, Ht., Wt., Hometown...","[{u'Pos.': u'IF', u'No.': u'1', u'Name_link': ...",2009-10,Baseball


In [183]:
# This is going to be somewhat space inefficient for now, but it's OK
rdf.to_pickle(EXPORT_ROSTERS)

### Next steps

- we have a 'played' database with links to bios
- we need to figure out what a unique player is
- we need to unify their metadata
- there is data to parse from each of their bios
- Maybe there is text analysis to be done from their season-to-season-development
- there is entity resolution to be done with hometowns and high schools.